Ebben a részben adatelőkészítést végzek


A feltételben beleírjuk, hogy akik az első évben elbuktak, azokat dobjuk el

In [2]:
# read the preprocessed data
import pandas as pd
df = pd.read_csv('preprocessed.csv')

In [18]:
subm = pd.read_csv('data_submission_example.csv')

In [3]:
df_test = df.copy()

In [5]:
valtozok = ['CONTRACT_DATE_OF_LOAN_AGREEMENT','BORROWER_ID','CONTRACT_INSTALMENT_AMOUNT','CONTRACT_INSTALMENT_AMOUNT_2','TARGET_EVENT_DAY','TARGET_EVENT','CONTRACT_ID','CONTRACT_LOAN_AMOUNT']

In [6]:
df_test = df_test[valtozok]

In [7]:
def datagen_modeA(origdf, startnap, endnap):
    df2 = origdf[origdf['CONTRACT_DATE_OF_LOAN_AGREEMENT']>=startnap].copy()
    df = df2[df2['CONTRACT_DATE_OF_LOAN_AGREEMENT']<endnap].copy()
    cust_df = pd.DataFrame({'BORROWER_ID':df['BORROWER_ID'].unique()})
    
    
    # TARGET VALTOZO 
    # a következő évben bekövetkező eseményeket nézzük
    # k: késedelmes lesz
    feltetel = (df['TARGET_EVENT_DAY']>endnap) & (df['TARGET_EVENT_DAY']<endnap+2*365) & (df['TARGET_EVENT'] == 'K')
    feltetel_2 = (df['TARGET_EVENT_DAY']<endnap) & (df['TARGET_EVENT_DAY']<startnap) & (df['TARGET_EVENT'] == 'K') #egy éven belül elbukott
    df['target'] = 0
    df.loc[feltetel,'target']=1
    df.loc[feltetel_2,'target']=2
    
    # drop rows where target is 2
    df = df[df['target']!=2]
    
    stat = df.groupby('BORROWER_ID',as_index=False).agg({'target':'max'})
    # akik nem fizettek az 1 lesz
    cust_df = cust_df.merge(stat,on='BORROWER_ID',how='left')
    # mindenki más aki előtörlesztett vagy semmi nem történt az 0
    cust_df['target'] = cust_df['target'].fillna(0)
    
    
    # BEMENO VALTOZOK
    # törlesztőösszeg
    stat = df.groupby("BORROWER_ID", as_index=False).agg({'CONTRACT_INSTALMENT_AMOUNT':'sum','CONTRACT_INSTALMENT_AMOUNT_2':'sum'})
    stat.columns=['BORROWER_ID','contract_instalment','contract_instalment_2']
    cust_df = cust_df.merge(stat,on='BORROWER_ID',how='left')
    
    
    
    # hany hitele van ÖSSZESEN
    
    stat = df.groupby("BORROWER_ID", as_index=False).agg({'CONTRACT_ID':'count','CONTRACT_LOAN_AMOUNT':'sum'})
    
    stat.columns=['BORROWER_ID','in_contract_num','in_loan_amount_sum']
    cust_df = cust_df.merge(stat,on='BORROWER_ID',how='left')
    
    return cust_df
    
    #cust_df = pd.DataFrame({'BORROWER_ID':df['BORROWER_ID'].unique()})
    
    #target_df = origdf[origdf['CONTRACT_DATE_OF_LOAN_AGREEMENT']>=endnap]
    #target_df = target_df[target_df['CONTRACT_DATE_OF_LOAN_AGREEMENT']<endnap+2*365]
    #target_df['target_int'] = target_df['TARGET_EVENT'].map({'-':0,'E':0,'K':1})
    #stat = target_df.groupby('BORROWER_ID',as_index=False).agg({'target_int':'max'})
    #cust_df = cust_df.merge(stat,on='BORROWER_ID',how='left')
    
    #return cust_df

R1 = datagen_modeA(df_test,0,365)
R2 = datagen_modeA(df_test,365*2,365*3)

In [8]:
R1['target'].mean()

0.012927453563396285

Modellépítés

In [9]:
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
from sklearn.metrics import log_loss

bemeno_valtozok = ['in_contract_num','in_loan_amount_sum','contract_instalment', 'contract_instalment_2']
R1 = R1.reset_index(drop=True)
cv=2
R1['fold'] = R1.index % cv
loglosslist = []
for i in range(cv):
    train_df = R1[R1['fold']!=i]
    test_df = R1[R1['fold']==i]
    gbm = GradientBoostingClassifier(n_estimators=100,max_depth=4,random_state=42)
    gbm.fit(train_df[bemeno_valtozok],train_df['target'])
    test_df['p'] = gbm.predict_proba(test_df[bemeno_valtozok])[:,1]
    logloss = log_loss(test_df['target'],test_df['p'])
    print(i,logloss)
    loglosslist.append(logloss)

C:\Users\Máté\AppData\Local\Temp\ipykernel_21080\1819506029.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['p'] = gbm.predict_proba(test_df[bemeno_valtozok])[:,1]


0 0.04942513427440773
1 0.04839209406846248


C:\Users\Máté\AppData\Local\Temp\ipykernel_21080\1819506029.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['p'] = gbm.predict_proba(test_df[bemeno_valtozok])[:,1]


In [12]:
gbm.fit(R1[bemeno_valtozok],R1['target'])

GradientBoostingClassifier(max_depth=4, random_state=42)

In [13]:
R2['p'] = gbm.predict_proba(R2[bemeno_valtozok])[:,1]

In [14]:
R2['p'].mean()

0.012633749047059807

In [15]:
R2['p2'] = 1-R2['p']
R2['p2'] = R2['p2']*(1-0.0148)/(R2['p2'].mean())
R2['p3'] = 1-R2['p2']

In [19]:
subm2 = subm.merge(R2[['BORROWER_ID','p3']],on = 'BORROWER_ID',how='left')
subm2.loc[ ~subm2['p3'].isna(),'PRED']=subm2.loc[ ~subm2['p3'].isna(),'p3']
del subm2['p3']

In [20]:
subm2['PRED'].mean()

0.014496131398390644

In [21]:
# RESULT
R2['p3'].mean()

0.014800000000000115

In [ ]:
subm2.to_csv("target_001.csv",index=None)